In [1]:
# Cloning GitHub repository for cloth segmentation by levindabhi

!git clone https://github.com/levindabhi/cloth-segmentation.git
!mv -T cloth-segmentation cloth_segmentation

Cloning into 'cloth-segmentation'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 62 (delta 5), reused 55 (delta 2), pack-reused 1
Unpacking objects: 100% (62/62), 16.84 MiB | 14.32 MiB/s, done.


In [2]:
!pip install timm
!pip install gdown
!pip install pytorch-metric-learning
!pip install boto3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 13.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 62.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.0 

In [3]:
import gdown

# Download pre-trained weights for the segmentation model (they're now gone, RIP)
gdown.download(id="1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ", output="cloth_segmentation/")

Downloading...
From (uriginal): https://drive.google.com/uc?id=1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ
From (redirected): https://drive.google.com/uc?id=1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ&confirm=t&uuid=cca5a4f2-f8e6-4b55-98ed-b0f640803220
To: /home/ubuntu/cloth_segmentation/cloth_segm_u2net_latest.pth
100%|██████████| 177M/177M [00:01<00:00, 103MB/s]  


'cloth_segmentation/cloth_segm_u2net_latest.pth'

In [5]:
from pathlib import Path
import os
import shutil
import gc
import pickle
import random
import json

from tqdm.auto import tqdm

import boto3
from botocore.client import Config

import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_pil_image

import timm
from pytorch_metric_learning import losses

from cloth_segmentation.utils.saving_utils import load_checkpoint_mgpu
from cloth_segmentation.networks import U2NET

import cv2

device = "cuda" if torch.cuda.is_available() else "cpu"

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
bucket_name = 's3-bucket'

# Requires secrets.json file with the S3 connection credentials in the current folder
with open('secrets.json', 'r') as j:
     secrets = json.load(j)
        
s3 = boto3.client(
    's3',
    aws_access_key_id=secrets['aws_access_key_id'],
    aws_secret_access_key=secrets['aws_secret_access_key'],
    config=Config(s3={'addressing_style': 'path'})
)

path_prefix = 'projects/street2shop/data/image_archives/'
response = s3.list_objects_v2(Bucket=bucket_name, Delimiter = '/', Prefix=path_prefix)
shop_zips_s3 = [prefix['Key'] for prefix in response['Contents'] if prefix['Key'] != path_prefix]
shop_zips_s3

In [8]:
# Custom transform to add 0 padding to square size for an image

class SquarePadTensor:
    def __call__(self, image):
        _, h, w = image.shape
        s = max(w, h)
        lft = (s - w) // 2
        rgt = s - w - lft
        top = (s - h) // 2
        bot = s - h - top

        padding = (lft, top, rgt, bot)
        return transforms.functional.pad(image, padding, 0, 'constant')

In [9]:
transform_seg = transforms.Compose([
        SquarePadTensor(),
        transforms.Resize(768),
        transforms.Normalize([0.5, 0.5, 0.5], 
                             [0.5, 0.5, 0.5])
    ])

transform_before_masking = transforms.Compose([
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])

transform_after_masking = transforms.Compose([
        transforms.Resize(224)
    ])

transforms_emb = [transform_before_masking, 
                  transform_after_masking]

In [10]:
class ImageDatasetSeg(Dataset):
    def __init__(self, dataframe, transform=None):
        self.images = dataframe["image_path"].values
        self.masks = dataframe["mask_path"].values
        
        self.to_tensor = transforms.Compose([transforms.ToTensor()])
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        path_to_img = self.images[idx]
        path_to_mask = self.masks[idx]
        
        image = cv2.imread(str(path_to_img).strip())
        try:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(path_to_img)
            raise e
            
        image = self.to_tensor(image).to(device)

        if self.transform:
            image = self.transform(image)
        return image, path_to_mask

In [11]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.img_labels = dataframe["label"].values
        self.images = dataframe["image_path"].values
        self.masks = dataframe["mask_path"].values
        
        self.to_tensor = transforms.Compose([transforms.ToTensor()])
        self.pad_for_masking = transforms.Compose([
            SquarePadTensor(),
            transforms.Resize(768)
        ])
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.img_labels[idx]
        
        image = cv2.imread(str(self.images[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        mask = cv2.imread(str(self.masks[idx]), cv2.IMREAD_GRAYSCALE)
        
        image = self.to_tensor(image).to(device)
        mask = self.to_tensor(mask).to(device)
        
        if not self.transform:
            image = self.pad_for_masking(image)
            masked_image = image * mask
            return masked_image, label
        elif isinstance(self.transform, list) and len(self.transform) == 2:
            image = self.transform[0](image)
            image = self.pad_for_masking(image)
            masked_image = image * mask
            masked_image = self.transform[1](masked_image)
            return masked_image, label
        else:
            image = self.pad_for_masking(image)
            masked_image = image * mask
            masked_image = self.transform(masked_image)
            return masked_image, label

In [ ]:
class EmbeddingsPipeline():
    def __init__(self, 
                 s3, 
                 bucket_name, 
                 model, 
                 model_seg, 
                 transform_emb,
                 transform_seg,
                 batch_size=256, 
                 batch_size_seg=8, 
                 files_root_folder="imgs",
                 s3_masked_zips_path="street2shop/data/image_archives_masked/",
                 s3_embeddings_path="street2shop/embeddings/"):
        self.s3 = s3
        self.bucket = bucket_name
        self.shop_s3_path = None
        self.s3_masked_zips_path = s3_masked_zips_path
        self.s3_embeddings_path = s3_embeddings_path
        
        self.root_path = Path(files_root_folder)
        self.root_path.mkdir(parents=True, exist_ok=True)

        self.shop_name = Path(shop_s3_path).stem
        self.shop_zip_filename = Path(shop_s3_path).name

        self.shop_path = self.root_path / self.shop_name
        self.zip_path = self.root_path / self.shop_zip_filename

        self.shop_imgs_path = self.shop_path / "images"
        self.shop_masks_path = self.shop_path / "masks"
        
        self.embeddings_path = None
        
        self.df = None
        self.df_filtered = None
        
        self.model = model
        self.model_seg = model_seg
        
        self.model.eval()
        self.model_seg.eval()
        
        self.transform_emb = transform_emb
        self.transform_seg = transform_seg
        
        self.batch_size_seg = batch_size_seg
        self.batch_size = batch_size

    def set_shop_s3_path(self, shop_s3_path):
        self.shop_s3_path = shop_s3_path
        
    def download_and_unpack_zip(self):
        if self.shop_s3_path:
            self.s3.download_file(self.bucket, str(self.shop_s3_path), str(self.zip_path))
            shutil.unpack_archive(self.zip_path, self.shop_imgs_path)
            self.zip_path.unlink()
        else:
            print("Path to image archive in S3 is not set")
        
    def create_dataframe(self):
        df_dict = {
            "label": [],
            "image_path": [],
            "mask_path": []
        }

        for hash_first_two in self.shop_imgs_path.iterdir():
            for hash_second_two in hash_first_two.iterdir():
                for img_path in hash_second_two.iterdir():
                    if img_path.stem != ".ipynb_checkpoints" and img_path.stat().st_size != 0:
                        df_dict["label"].append(img_path.stem)
                        df_dict["image_path"].append(str(img_path))
                        df_dict["mask_path"].append(str(img_path).replace(str(self.shop_imgs_path), str(self.shop_masks_path)))

        self.df = pd.DataFrame.from_dict(df_dict)
    
    def create_masks(self):
        if not self.df:
            return "Create dataframe first"
        
        dataset_seg = ImageDatasetSeg(self.df, transform=self.transform_seg)
        dataloader_seg = DataLoader(self.dataset_seg, batch_size=self.batch_size_seg, shuffle=False)

        self.model_seg.eval()
        for data, path_to_mask in tqdm(dataloader_seg):
            data = data.to(device)
            with torch.no_grad():
                output = self.model_seg(data)
                output = F.log_softmax(output[0], dim=1)
                output = torch.max(output, dim=1, keepdim=True)[1].bool().float()   
                output = output.detach().cpu()
                for i in range(output.shape[0]):
                    Path(path_to_mask[i]).parent.mkdir(parents=True, exist_ok=True)
                    to_pil_image(output[i], "L").save(path_to_mask[i])

        torch.cuda.empty_cache()
        gc.collect()
        
    def upload_masks_to_s3(self):
        shutil.make_archive(self.shop_path, 'zip', self.shop_path)  # Makes zipfile {root_path}/{shop_name}.zip from folder {root_path}/{shop_name}
        self.s3.upload_file(Filename=str(self.zip_path), Bucket=self.bucket, Key=self.s3_masked_zips_path + self.shop_zip_filename)
        self.zip_path.unlink()
    
    @staticmethod
    def __calculate_mask_fillrate(row):
        mask = cv2.imread(str(row), cv2.IMREAD_GRAYSCALE)
        pixels = mask.shape[0] * mask.shape[1]

        fillrate = round(mask.sum() / 255 / pixels, 3)
        return fillrate

    def filter_df(self):
        df_filtered = self.df.copy()
        df_filtered["mask_fillrate"] = df_filtered["mask_path"].apply(self.__calculate_mask_fillrate)
        df_filtered = df_filtered[df_filtered.mask_fillrate >= 0.01].reset_index(drop=True).copy()
        self.df_filtered = df_filtered
        
    def create_embeddings(self):
        if self.df_filtered:
            df = self.df_filtered
        elif :
            print("Warning! Filtered dataframe wasn't created, using unfiltered dataframe")
            df = self.df
        else:
            return "Create and filter dataframe first"
            
        dataset = ImageDataset(df, transform=self.self.transform_emb)
        dataloader = DataLoader(dataset, batch_size=self.self.batch_size, shuffle=False)

        embeddings = {}

        self.model.eval()
        for data, label in tqdm(dataloader):
            data = data.to(device)

            with torch.no_grad():
                output = model(data).detach().cpu().numpy()
                for i in range(output.shape[0]):
                    embeddings[label[i]] = output[i]

        self.embeddings_path = self.root_path / f"{self.shop_name}_embeddings.pickle"
        with open(self.embeddings_path, "wb") as f:
            pickle.dump(embeddings, f)
            
        torch.cuda.empty_cache()
        gc.collect()
            
    def upload_embeddings(self):
        if self.embeddings_path:
            s3.upload_file(Filename=str(self.embeddings_path), Bucket=self.bucket, Key=self.s3_embeddings_path + self.embeddings_path.name)
        else:
            print("Embeddings file wasn't created")
            
    def cleanup_images(self):
        shutil.rmtree(self.shop_path)

In [19]:
checkpoint_path = "cloth_segmentation/cloth_segm_u2net_latest.pth"

net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)

for param in net.parameters():
    net.requires_grad = False

net.to(device)
net.eval()
pass

----checkpoints loaded from path: cloth_segmentation/cloth_segm_u2net_latest.pth----


In [20]:
model_path = "model_checkpoint_train_epoch_50.pt"

model = timm.create_model("ig_resnext101_32x8d", pretrained=False)
model.train()

model.fc = nn.Linear(in_features=2048, out_features=512)

model.load_state_dict(torch.load(model_path))

for param in model.parameters():
    param.requires_grad = False

model.to(device)
model.eval()
pass

In [ ]:
imgs_root = "imgs"
s3_masked_zips_path = "projects/street2shop/data/image_archives_masked/"
s3_embeddings_path = "projects/street2shop/embeddings/"

batch_size_seg = 24
batch_size = 512

pipeline = EmbeddingsPipeline(s3,
                              bucket_name, 
                              model, 
                              model_seg, 
                              transform_emb,
                              transform_seg,
                              batch_size=batch_size,
                              batch_size_seg=batch_size_seg,
                              files_root_folder=imgs_root,
                              s3_masked_zips_path=s3_masked_zips_path)

for shop_zip_s3 in tqdm(shop_zips_s3):
    pipeline.set_shop_s3_path(shop_zip_s3)
    pipeline.download_and_unpack_zip()
    pipeline.create_dataframe()
    pipeline.create_masks()
    pipeline.upload_masks_to_s3()
    pipeline.filter_df()
    pipeline.create_embeddings()
    pipeline.upload_embeddings()
    pipeline.cleanup_images()